In [ ]:
# import required  libraries
import numpy as  np 
import pandas as pd 

In [ ]:
#  import the dataset from the local directory
df1= pd.read_csv("/Users/mac/Desktop/Project/movie_recommender_project/tmdb_movies_5k.csv")
df2 =pd.read_csv("/Users/mac/Desktop/Project/movie_recommender_project/tmdb_movie_7k.csv")


In [ ]:
df1.shape ,df2.shape

In [ ]:
df1.head()

In [ ]:
df2.head()

In [ ]:
# print the first 5 movies
movies = pd.concat([df1, df2], axis=0, ignore_index=True)



In [ ]:
# shape of the movie data
movies.shape


In [ ]:
# print the first 5 row of movies dataset
movies.head()

In [ ]:
# print the columns of the data 
movies.columns

In [ ]:
# we only use these following columns for the my project 
"""
genres
movie_id
keywords
title
overview
cast 
crew 
"""
# update the movies dataset only having upper   these columns

movies = movies[['movie_id','title','genres','keywords','overview','cast','crew']]

In [ ]:
movies.head()

In [ ]:
movies['crew'][0:10]

In [ ]:
movies['genres'].unique()

In [ ]:
# checking for null values:
df1.isnull().sum()

In [ ]:
# drop the null values row
movies.dropna(inplace= True)
movies.isnull().sum()

In [ ]:
movies.shape

In [ ]:
# checking the duplicate values
movies.duplicated().sum()

In [ ]:
# sample seeing the genres  column data 
movies.iloc[9].genres

In [ ]:
# fetch data Director ,writer , producer  from crew column
import pandas as pd
import re

# Sample: if your df['crew'] has missing or NaN entries
movies['crew'] = movies['crew'].fillna('')

# Roles you want to extract
roles = ['Director', 'Producer', 'Writer']

# Loop over each role and extract matching names
for role in roles:
    movies[role.lower() + 's'] = movies['crew'].apply(
        lambda x: ', '.join(re.findall(rf'(.*?)(?= \({role}\))', x)) if role in x else '')

In [ ]:
# Convert list fields to joined strings
movies['genres']   = movies['genres'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
movies['keywords'] = movies['keywords'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
movies['cast']     = movies['cast'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
movies['crew']     = movies['crew'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

movies.head()

In [ ]:
# Create unified "tags" for sentence-transformers
movies['tags'] = (
    movies['overview'].fillna('') + ' | ' +
    movies['genres'].fillna('') + ' | ' +
    movies['keywords'].fillna('') + ' | ' +
    movies['cast'].fillna('') + ' | ' +
    movies['crew'].fillna('')
)

movies['tags']

In [ ]:


# new dataframe with movie_id ,title, and tags only as 'new_df'
new_df = movies[['movie_id','title','tags']]

# print  the new dataframe
new_df.head()

In [ ]:
# Lowercase the tags for consistency (important for text-based similarity)
new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())

In [ ]:
new_df['tags']

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load the transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')  # small & fast, good accuracy

# Encode tags to embeddings
embeddings = model.encode(new_df['tags'].tolist(), show_progress_bar=True)

# Compute similarity
similarity = cosine_similarity(embeddings)


In [ ]:
similarity.shape

In [ ]:
# reset index 
new_df = new_df.reset_index(drop=True)

In [ ]:
def recommend(movie_title):
    movie_index = new_df[new_df['title'] == movie_title].index[0]
    distances = list(enumerate(similarity[movie_index]))
    movies_list = sorted(distances, key=lambda x: x[1], reverse=True)[1:6]

    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [ ]:
print(similarity.shape)
print(new_df.shape)


In [ ]:
recommend("The Dark Knight")

In [ ]:
# print the similarity score of the first vector with the rest  vectors
similarity[3]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Example: check similarity between The Dark Knight and recommended movie
i = new_df[new_df['title'] == "The Dark Knight"].index[0]
similarity_scores = cosine_similarity([embeddings[i]], embeddings)[0]
top_indices = similarity_scores.argsort()[-6:-1][::-1]

for idx in top_indices:
    print(f"{new_df.iloc[idx]['title']} — Similarity: {similarity_scores[idx]:.4f}")

  

In [ ]:
# check the my model work :
recommend('Ariel')

In [ ]:
#  save the movie dictionary as pickle file
import  pickle
pickle.dump(new_df.to_dict(), open('movie_dict.pkl','wb'))

In [ ]:
# save the similarity matrix as pickle file:
pickle.dump(similarity , open('similarity.pkl','wb'))